In [ ]:
#pip
#pip install azure-cognitiveservices-speech==1.24.0

In [ ]:
from dotenv import load_dotenv
from datetime import datetime
import os

In [ ]:
#import namespace
import azure.cognitiveservices.speech as speech_sdk

In [ ]:
def main():
    try:
        global cog_key
        global speech_config
        global translation_config
        load_dotenv()
        cog_key=os.getenv("COG_SERVICE_KEY")
        cog_region=os.getenv("COG_SERVICE_REGION")

        #Configure translation
        translation_config = speech_sdk.translation.SpeechTranslationConfig(cog_key,cog_region)
        translation_config.speech_recognition_language = 'en-US'
        translation_config.add_target_language('fr')
        translation_config.add_target_language('es')
        translation_config.add_target_language('hi')
        print('Readu tp translate from',translation_config.speech_recognition_language)

        #Configure speech
        speech_config = speech_sdk.SpeechConfig(cog_key, cog_region)
        
        #Get user input
        targetLanguage = ''
        while targetLanguage != "quit":
            targetLanguage = input('\nEnter a target language\n fr = French\n es = Spanish\n hi = Hindi\n Enter anything else to stop').lower()
            if targetLanguage in translation_config.target_languages:
                Translate(targetLanguage)
            else:
                targetLanguage = 'quit'


    except Exception as ex:
        print(ex)


In [ ]:
def Translate(targetLanguage):
    translation = ''

    # Translate speech
    audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
    translator = speech_sdk.translation.TranslationRecognizer(translation_config, audio_config)
    print("Speak now...")
    result = translator.recognize_once_async().get()
    print('Translating "{}"'.format(result.text))
    translation = result.translations[targetLanguage]
    print(translation)

    # Synthesize translation
    voices = {
        "fr":"fr-FR-HenriNeural",
        "es":"es-ES-ElviraNeural",
        "hi":"hi-IN-MadhurNeural"
    }
    speech_config.speech_synthesis_voice_name = voices.get(targetLanguage)
    speech_synthesizer = speech_sdk.SpeechSynthesizer(speech_config)
    speak = speech_synthesizer.speak_text_async(translation).get()
    if speak.reason != speech_sdk.ResultReason.SynthesizingAudioCompleted:
        print(speak.reason)

In [ ]:
if __name__ == "__main__":
    main()